In [1]:
from nav2_simple_commander.robot_navigator import BasicNavigator
import rclpy

rclpy.init()
nav = BasicNavigator()

In [2]:
nav.waitUntilNav2Active()

[INFO] [1714368030.035300416] [basic_navigator]: Nav2 is ready for use!


In [13]:
from geometry_msgs.msg import PoseStamped

goal_pose = PoseStamped()
goal_pose.header.frame_id = 'map'
goal_pose.header.stamp = nav.get_clock().now().to_msg()
goal_pose.pose.position.x = 6.841780662536621
goal_pose.pose.position.y = -5.175563335418701
goal_pose.pose.position.z = 0.002471923828125
goal_pose.pose.orientation.x = 0.0
goal_pose.pose.orientation.y = 0.0
goal_pose.pose.orientation.z = 0.0
goal_pose.pose.orientation.w = 1.0

In [14]:
nav.goToPose(goal_pose)

[INFO] [1714368131.175877174] [basic_navigator]: Navigating to goal: 6.841780662536621 -5.175563335418701...


True

In [5]:
from rclpy.duration import Duration

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining: ' + '{:.2f}'.format(
            feedback.distance_remaining) + ' meters.')
        
        # Some navigation timeout to demo cancellation
        if Duration.from_msg(feedback.navigation_time) > Duration(seconds=10.0):
            nav.cancelTask()

Distance remaining: 2.06 meters.
Distance remaining: 1.99 meters.
Distance remaining: 1.84 meters.
Distance remaining: 1.70 meters.
Distance remaining: 1.60 meters.
Distance remaining: 1.46 meters.
Distance remaining: 1.39 meters.
Distance remaining: 1.21 meters.
Distance remaining: 1.14 meters.
Distance remaining: 1.04 meters.
Distance remaining: 0.90 meters.
Distance remaining: 0.71 meters.
Distance remaining: 0.64 meters.
Distance remaining: 0.57 meters.
Distance remaining: 0.43 meters.
Distance remaining: 0.31 meters.
Distance remaining: 0.24 meters.


[INFO] [1714368071.046217376] [basic_navigator]: Canceling current task.


In [6]:
from nav2_simple_commander.robot_navigator import TaskResult

result = nav.getResult()
if result == TaskResult.SUCCEEDED:
    print('Goal succeeded!')
elif result == TaskResult.CANCELED:
    print('Goal was canceled!')
elif result == TaskResult.FAILED:
    print('Goal failed!')

Goal was canceled!


In [7]:
import rclpy
from geometry_msgs.msg import PoseWithCovarianceStamped

# rclpy.init()
pose_current = PoseWithCovarianceStamped()
pose_current

geometry_msgs.msg.PoseWithCovarianceStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id=''), pose=geometry_msgs.msg.PoseWithCovariance(pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)), covariance=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])))

In [8]:
pose_current.pose.pose.position

geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0)

In [9]:
pose_current.pose.pose.orientation

geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)

In [10]:
pose_node = rclpy.create_node('sub_pose')

def callback(data):
    global pose_current
    print("===")
    print(data.pose.pose.position)
    print(data.pose.pose.orientation)
    pose_current = data

In [11]:
pose_node.create_subscription(PoseWithCovarianceStamped, '/amcl_pose', callback, 10)

In [15]:
rclpy.spin_once(pose_node)

In [16]:
from tf_transformations import euler_from_quaternion

In [17]:
pose_current

geometry_msgs.msg.PoseWithCovarianceStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id=''), pose=geometry_msgs.msg.PoseWithCovariance(pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)), covariance=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])))

In [18]:
pose_current.pose.pose.orientation

geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)

In [19]:
q = [0, 0, 0, 0]
q[0] = pose_current.pose.pose.orientation.x
q[1] = pose_current.pose.pose.orientation.y
q[2] = pose_current.pose.pose.orientation.z
q[3] = pose_current.pose.pose.orientation.w

euler_from_quaternion(q)

(0.0, -0.0, 0.0)

In [20]:
import numpy as np

def convert_degree(input):
    return np.array(input) * 180. / np.pi

convert_degree(euler_from_quaternion(q))

array([ 0., -0.,  0.])

In [46]:
from tf_transformations import quaternion_from_euler

to_radian = np.pi / 180.
tmp = [0, 0, 0] # 오일러 각도
print(np.array(tmp)*to_radian)

quaternion_from_euler(tmp[0], tmp[1], tmp[2])

[0.         0.         0.08726646]


(-0.0, 0.0, 0.5984721441039565, -0.8011436155469337)

In [47]:
from nav2_simple_commander.robot_navigator import BasicNavigator
from geometry_msgs.msg import PoseStamped
nav = BasicNavigator()

target = quaternion_from_euler(tmp[0], tmp[1], tmp[2])
goal_pose = PoseStamped()

goal_pose.header.frame_id = 'map'
goal_pose.header.stamp = nav.get_clock().now().to_msg()
goal_pose.pose.position.x = 2.915587902069092
goal_pose.pose.position.y = -3.6941566467285156
goal_pose.pose.position.z = 0.002471923828125
goal_pose.pose.orientation.x = target[0]
goal_pose.pose.orientation.y = target[1]
goal_pose.pose.orientation.z = target[2]
goal_pose.pose.orientation.w = target[3]

[WARN] [1714370724.096966132] [rcl.logging_rosout]: Publisher already registered for provided node name. If this is due to multiple nodes with the same name then all logs for that logger name will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.


In [48]:
from rclpy.duration import Duration

nav.goToPose(goal_pose)

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining: ' + '{:.2f}'.format(
            feedback.distance_remaining) + ' meters.')
        
        # Some navigation timeout to demo cancellation
        # 네비게이션이 실행되고 10초가 지나면 실행 취소
        if Duration.from_msg(feedback.navigation_time) > Duration(seconds=10.0):
            nav.cancelTask()

[INFO] [1714370726.384801631] [basic_navigator]: Navigating to goal: 2.915587902069092 -3.6941566467285156...


Distance remaining: 0.19 meters.
Distance remaining: 0.19 meters.
Distance remaining: 0.19 meters.
Distance remaining: 0.19 meters.
Distance remaining: 0.14 meters.
Distance remaining: 0.14 meters.
Distance remaining: 0.19 meters.
Distance remaining: 0.19 meters.
